In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
LOCAL_RETROFUNDING_DIR = "../../../../../../../../GitHub/Retro-Funding/"

In [3]:
def stringify(arr):
    return "'" + "','".join(arr) + "'"

In [4]:
df_deps = pd.read_csv(LOCAL_RETROFUNDING_DIR + 'eval-algos/S7/data/M1/devtooling/project_dependencies.csv')
df_devs = pd.read_csv(LOCAL_RETROFUNDING_DIR + 'eval-algos/S7/data/M1/devtooling/developers_to_projects.csv')
df_onchain = pd.read_csv(LOCAL_RETROFUNDING_DIR + 'eval-algos/S7/data/M1/devtooling/onchain_projects.csv')

In [5]:
PROJECT_IDS = df_onchain['project_id'].unique()

In [6]:
START_DATE = '2025-02-01'
END_DATE = '2025-03-01'
METRICS = [
    f"{c.upper()}_gas_fees_daily"
    for c in [  # Supported chains
        'automata', 'cyber', 'ham', 
        'base', 'bob', 'frax', 'ink', 'kroma', 'lisk', 'lyra',
        'metal', 'mint', 'mode', 'optimism', 'orderly', 'polynomial',
        'race', 'redstone', 'shape', 'soneium', 'swan', 'swell',
        'unichain', 'worldchain', 'xterio', 'zora'
    ]
]

df_onchain_metrics = client.to_pandas(f"""
    select
        tm.project_id,
        SUM(tm.amount) as amount
    from oso.timeseries_metrics_by_project_v0 tm
    join oso.metrics_v0 m on tm.metric_id = m.metric_id
    where    
        tm.sample_date >= DATE('{START_DATE}') AND tm.sample_date < DATE('{END_DATE}')    
        and tm.project_id IN ({stringify(PROJECT_IDS)})
        and m.metric_name IN ({stringify(METRICS)})
    group by 1
    order by 2 desc
""")

df_onchain_metrics.head()

,project_id,amount
0,NMwxMeqwLAUieJgJHRyjvE6q/xw/KfE7em6oi9U1aRc=,1105.961558
1,SrAW7CxbX2q6KzPf1xXwJsc5IQZqIkDDOkAx3/nTS0M=,664.116864
2,pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ=,383.060575
3,ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=,187.705441
4,L1FGc2AIMWcbDulPkpgzndWnljElZEAxxr+jPOdn0WQ=,72.229524


In [7]:
df_mapping = client.to_pandas(f"""
    select distinct
        p2p.*,
        abp.artifact_source,
        abp.artifact_namespace,
        abp.artifact_name        
    from int_projects_to_projects as p2p
    join int_artifacts_by_project_all_sources as abp on p2p.artifact_id = abp.artifact_id
    join projects_by_collection_v1 pbc on p2p.external_project_id = pbc.project_id
    where
        abp.artifact_type in ('REPOSITORY', 'DEFILLAMA', 'DEPLOYER')
        and p2p.ossd_project_id in ({stringify(PROJECT_IDS)})
        and pbc.collection_name = '8-1'
""")

In [8]:
dups = df_mapping.groupby(['ossd_project_id', 'ossd_project_name'])['external_display_name'].nunique()
dups = list(dups[dups>1].reset_index()['ossd_project_id'])
(
    df_mapping[df_mapping['ossd_project_id'].isin(dups)]
    [['ossd_project_name', 'external_project_name', 'external_display_name']]
    .drop_duplicates()
    .sort_values(by='ossd_project_name')
)

,ossd_project_name,external_project_name,external_display_name
154,flayerlabs,0x259b729af86ba38c0c84be9b739dfbca109a6df98537...,Flaunch
259,flayerlabs,0xaa1b878800206da24ee7297fb202ef98a6af0fb3ec29...,Stepan's project
187,gfx-labs,0x9d9162fac637991d2a56da8fbe03dba24c87eb82d47c...,Uniswap on Superchain (Oku)
395,gfx-labs,0xb926ab918147d2264dba94a429eb07986aa533fd6062...,Oku Trade
41,party-dao,0x850df51e29f2846a5f085d88e6b6fc13fad51ad7161f...,Party.app + Party Protocol
48,party-dao,0x9fb539aa3ddb63cda1c29b50fbeaa6071f290926ce10...,UNO
68,party-dao,0xd774790974f07c7c5130903d47ae885f0f3415f0c8ee...,create.party.app
423,party-dao,0x40ee6048c99aefd288f6133db207849d0951fe6b62af...,ORO
5,velodrome,0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be...,Velodrome Finance
46,velodrome,0x0008577196fa6ec286440b418e2f6305fc10e62ce759...,CreateX


In [9]:
oso_project_names = df_onchain.set_index('project_id')['project_name'].to_dict()
atlas_project_names = df_mapping.set_index('external_project_name')['external_display_name'].drop_duplicates().to_dict()
oso_to_atlas_ids = df_mapping.groupby('ossd_project_id')['external_project_name'].apply(lambda x: list(x.unique()))
atlas_name_to_oso_id = df_mapping.set_index('external_project_name')['external_project_id'].to_dict()
project_gas_fees = df_onchain_metrics.set_index('project_id')['amount'].to_dict()

In [10]:
df_results = pd.read_json(LOCAL_RETROFUNDING_DIR + 'results/S7/m1_devtooling_results.json')

df_results['onchain_builder_oso_project_ids_2'] = df_results['onchain_builder_oso_project_ids'].apply(
    lambda lst_of_ids: [oso_id for oso_id in lst_of_ids if oso_id in project_gas_fees]
)
df_results['onchain_builder_op_atlas_ids_2'] = df_results['onchain_builder_oso_project_ids_2'].apply(
    lambda oso_lst: [atlas_id for atlas_lst in oso_lst for atlas_id in oso_to_atlas_ids.get(atlas_lst,[])]
)
df_results['onchain_builder_oso_project_names_2'] = df_results['onchain_builder_oso_project_ids_2'].apply(
    lambda lst_of_ids: sorted([oso_project_names[oso_id] for oso_id in lst_of_ids if oso_id in oso_project_names])
)
df_results['onchain_builder_op_atlas_names_2'] = df_results['onchain_builder_op_atlas_ids_2'].apply(
    lambda lst_of_ids: sorted([atlas_project_names[atlas_id] for atlas_id in lst_of_ids if atlas_id in atlas_project_names])
)
df_results['downstream_gas'] = df_results['onchain_builder_oso_project_ids_2'].apply(
    lambda lst_of_ids: sum([project_gas_fees[oso_id] for oso_id in lst_of_ids if oso_id in project_gas_fees])
)
df_results = df_results[[
    'project_id', 'project_name', 'display_name',
    'round_id', 'is_eligible', 'op_reward',
    'star_count', 'fork_count', 'num_packages_in_deps_dev',
    'package_connection_count', 'developer_connection_count',
    'downstream_gas', 'trusted_developer_usernames', 
    
    'onchain_builder_op_atlas_names',
    'onchain_builder_op_atlas_names_2', 'onchain_builder_oso_project_names_2',
    'onchain_builder_oso_project_ids', 'onchain_builder_oso_project_ids_2',    
    'onchain_builder_op_atlas_ids', 'onchain_builder_op_atlas_ids_2'
]]       
df_results.tail()

,project_id,project_name,display_name,round_id,is_eligible,op_reward,star_count,fork_count,num_packages_in_deps_dev,package_connection_count,developer_connection_count,downstream_gas,trusted_developer_usernames,onchain_builder_op_atlas_names,onchain_builder_op_atlas_names_2,onchain_builder_oso_project_names_2,onchain_builder_oso_project_ids,onchain_builder_oso_project_ids_2,onchain_builder_op_atlas_ids,onchain_builder_op_atlas_ids_2
146,ZYiGfGhLr1kdrN5ykW1QF/4yUjZPQM/71bkaYtunHEc=,0xf2a60464d2a56fb47d2f8c13001edea71eda11ffd8ff...,Unruggable,7,False,0.00,71,5,0,0,4,8.133491,"[mo-hussain, yorhodes, 0xpenryn, pcaversaccio]",[],[],"[hyperlane-xyz, worldcoin]","[Jpscljo0tkzVDYKwVNnTSuR+AFUyy4EHg4AWd+Mdnd4=,...","[Jpscljo0tkzVDYKwVNnTSuR+AFUyy4EHg4AWd+Mdnd4=,...",[],[]
147,UZSaL/0UXqdDxpqfSpbeIu3OmDIpdUn8wjgU+TWzKbw=,0xf839a585342327848d4541a6fcc315404e879537a60a...,Dappnode,7,True,1342.02,76,26,3,0,8,1055.347996,"[mikeki, shanefontaine, michael-a-heuer, joaqu...","[Aragon OSx, Exa App, Hop Protocol, thirdweb]","[Exa App, Hop Protocol, Uniswap on Superchain ...","[aragon, exactly, hop-protocol, metamask, thir...","[SrAW7CxbX2q6KzPf1xXwJsc5IQZqIkDDOkAx3/nTS0M=,...","[SrAW7CxbX2q6KzPf1xXwJsc5IQZqIkDDOkAx3/nTS0M=,...",[0xc6052138bbdae5976fa2866f46b0537182d4126c4bb...,[0x9d9162fac637991d2a56da8fbe03dba24c87eb82d47...
148,dk1gwy0G/dP9SOcyolL+bfgPQJ0ngqXF6ro9qyQdGwo=,0xf9ebb0464e7162627ab3de5fe03a21c879e932233e14...,Air3,7,False,0.00,2,0,0,0,0,0.000000,[],[],[],[],[],[],[],[]
149,NGbZU3ScsbMUnmgUnRJvqBXFPRljZDbDYozQSJw1Vh0=,0xfa7d950cfda2c634b052b6c07e75daaa95dd22ac1f22...,Solhint,7,True,56487.95,110,32,4,121,30,2689.359286,"[scott-silver, noisekit, cruzdanilo, pcaversac...","[Aave, Account Abstraction - ERC-4337, Across ...","[Aave, Account Abstraction - ERC-4337, Across ...","[0x-splits, 0xproject, 0xsend, 0xsequence, 1-i...","[0rInH1b9Mh04BAwmikdFhcH/YDDR7xg7vxtho7SyKsY=,...","[0rInH1b9Mh04BAwmikdFhcH/YDDR7xg7vxtho7SyKsY=,...",[0x4288aacdda65af69c03b2c9b9d057e2b93ed1b7671e...,[0x4288aacdda65af69c03b2c9b9d057e2b93ed1b7671e...
150,g1pi0NuLcPtSAjpxGRMTIM2ZRs5y6h3IpcW0PRKvSQo=,0xfc54b4a9c537be25238cba3f05100b733aeee83a3eb3...,Multicaller,7,True,3904.57,68,11,1,0,18,25.044595,"[zeframlou, stephankmin, pcaversaccio, greg-sc...","[Morpho , PHI, Rainbow (Rainbow Router), Sabli...","[Multicaller, PHI, Rainbow (Rainbow Router), S...","[0xsend, bunniapp, curve, fraxfinance, hyperla...","[fhQOM88bZld6udOIUL5hyBRrmGm4mv7G7sv3Opl6/7g=,...","[fhQOM88bZld6udOIUL5hyBRrmGm4mv7G7sv3Opl6/7g=,...",[0x5614d129eba060ce7d224e8e8f9f29aeddfdb572b0d...,[0x5614d129eba060ce7d224e8e8f9f29aeddfdb572b0d...


In [11]:
# json_str = df_results.to_json(orient='records', indent=2)
# clean_json_str = json_str.replace('\\/', '/')
# with open('m1_devtooling_results_v2.json', 'w') as file:
#     file.write(clean_json_str)

In [12]:
data = []
for (oso_id, atlas_id, atlas_name), row in df_results.set_index(['project_id', 'project_name', 'display_name']).iterrows():
    for builder in row['onchain_builder_oso_project_ids']:
        if builder not in project_gas_fees:
            continue
        gas = project_gas_fees[builder]
        atlas_ids = oso_to_atlas_ids.get(builder,[])
        atlas_names = " | ".join([atlas_project_names[_id] for _id in atlas_ids])
        atlas_ids_mapped_to_oso_ids = " | ".join([atlas_name_to_oso_id[_id] for _id in atlas_ids])
        data.append({
            'devtool_oso_id': oso_id,
            'devtool_atlas_id': atlas_id,
            'devtool_name': atlas_name,
            'builder_oso_id': builder,
            'builder_oso_slug': oso_project_names.get(builder),
            'builder_atlas_ids': " | ".join(atlas_ids),
            'builder_atlas_id_mapped_to_oso_id': atlas_ids_mapped_to_oso_ids,
            'builder_atlas_names': atlas_names,
            'builder_gas_fees': gas
        })

In [13]:
df = pd.DataFrame(data)
#df.to_csv("m1_devtooling_downstream_gas.csv")